# Explore, segment, and cluster the neighborhoods in the city of Toronto  
For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto.  
I will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format. Once the data is in a structured format, I can apply the analysis to explore and cluster the neighborhoods in the city of Toronto.  
### Sorry for my Engish!
---

## Table of Contents  

1. <a href="#item1">Cooking neighborhoods data</a>  
  
2. <a href="#item2">Define neighborhoods location</a>  
  
3. <a href="#item3">Cluster heighborhoods in the city of Toronto</a>  
  
4. <a href="#item4">Examine Clusters</a>  
  

---


<a id='item1'></a>

## 1. Cooking neighborhoods data

In [34]:
import pandas as pd
import numpy as np

#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 0)
pd.set_option('display.show_dimensions',True)

print('Pandas version: ',pd.__version__)
print('Numpy version: ',np.__version__)

Pandas version:  0.24.1
Numpy version:  1.15.4


Install packages *lxml* and *BeautifulSoup* for parsing HTML pages if you haven't installed yet.

In [2]:
# install lxml
#!conda install -c conda-forge lxml --yes

# install BeautifulSoup
#!conda install -c conda-forge BeautifulSoup4 --yes

Let's use *BeautifulSoup* package to parse wiki page. You should be aware that the page contains several tables and I will use *find()* method instead of *findAll()*. Lucky me, the neighborhood data is contained in the first table on the page. So, the first *find()* gives us all that we want.

In [2]:
# import libraries
from urllib.request import urlopen
import bs4

#wiki page for neighborhoods in Toronto
WIKI_PAGE = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# array for table content
data = []

# get contents of wiki page
page = urlopen(WIKI_PAGE)
# init soup
soup = bs4.BeautifulSoup(page, "lxml")
# get all content inside _first_ 'tbody' tag
table_body = soup.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append(cols)

# because of first row of table contains header names arranged by 'th' tags 
# first element of data array is empty - just remove it 
del data[0]

# check result
data[0:5]

[['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront']]

All looks good. Convert array to pandas dataframe:

In [3]:
df = pd.DataFrame(data, columns=['PostalCode','Borough','Neighborhood'])
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [4]:
df.shape

(287, 3)

Remove all rows where borough is not assigned:

In [5]:
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace=True)
df.shape

(210, 3)

If we have 'not assigned' neighborhood we use borough name instead. Let's look how many such rows exists.

In [6]:
df[df['Neighborhood'] == 'Not assigned']

,PostalCode,Borough,Neighborhood
7,M7A,Queen's Park,Not assigned


Just one. Anyway I use lambda function applied to dataframe which return either neighborhood or borough if neighborhood is not assigned.

In [7]:
df['Neighborhood'] = df.apply(lambda x : x['Borough'] if x['Neighborhood'] == 'Not assigned' else x['Neighborhood'], axis=1)
df[df['Neighborhood'] == 'Not assigned']

,PostalCode,Borough,Neighborhood


Done! All neighborhoods assigned.  
More than one neighborhood can exist in one postal code area. I'll group dataframe by postal code and borough, then apply lambda function to concatenate neighborhoods in every group:

In [8]:
df_neighborhood = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda x: "%s" % ', '.join(x)).to_frame().reset_index()
df_neighborhood.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


At the end of part 1 print number of records of dataset:

In [9]:
df_neighborhood.shape

(103, 3)

<a id='item2'></a>

## 2. Define neighborhoods location
Now that we have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.  
Because of Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so I will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.  


In [12]:
# install it
#!conda config --set ssl_verify false # to avoid ssl issue
!conda install -c conda-forge geocoder --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.4 MB

The following NEW packages will be INSTALLED:

    geocoder:        1.38.1-py_1       conda-forge
    ratelim:         0.1.6-py_2        conda-forge

The following packages will be UPDATED:

    

I create function to get the geographical coordinates.

In [13]:
import geocoder

def coordinates(postal_code):
    # initialize variable to None
    lat_lng_coords = None
    
    # loop until we get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        
    return lat_lng_coords[0], lat_lng_coords[1]

Through *map* function I generate geo coordinates for every postal code in dataset:

In [14]:
df_neighborhood['Latitude'], df_neighborhood['Longitude'] = zip(*df_neighborhood['PostalCode'].map(coordinates))
df_neighborhood.head(10)

KeyboardInterrupt: 

Something went wrong... After 10 minutes I can't get coordinates through *geocoder* package. Well, I use anoyher way - a csv file that has the geographical coordinates of each postal code. Link is http://cocl.us/Geospatial_data. 

In [15]:
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Let's merge two dataframes into one based of postal code:

In [16]:
df_neighborhood = df_neighborhood.merge(df_geo, how='left', left_on='PostalCode', right_on='Postal Code')
df_neighborhood.head(10)

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,M1J,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",M1K,43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",M1L,43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",M1M,43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",M1N,43.692657,-79.264848


Drop duplicate column:

In [17]:
df_neighborhood.drop('Postal Code', axis=1, inplace=True)
df_neighborhood.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


To be sure that merging was successful I use *describe* method to get some statistics:

In [18]:
df_neighborhood.describe()

,Latitude,Longitude
count,103.000000,103.000000
mean,43.704608,-79.397153
std,0.052463,0.097146
min,43.602414,-79.615819
25%,43.660567,-79.464763
50%,43.696948,-79.388790
75%,43.745320,-79.340923
max,43.836125,-79.160497


Well, as we see from *count*, *min* and *max* numbers there are no missing values exist.  
  
#### As the end of part two let's look on neighborhood dataframe:

In [19]:
df_neighborhood.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


<a id='item3'></a>

## 3. Clustering neighborhoods in the city of Toronto
I will use the explore function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. I will use the k-means clustering algorithm to complete this task. Finally, I will use the Folium library to visualize the neighborhoods in city of Toronto and their emerging clusters.

In [21]:
# import libraries
import json # library to handle JSON files

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't installed it yet
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         868 KB

The following NEW packages will be INSTALLED:

    altair:  3.2.0-py36_0 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge


vincent-0.4.4        | 28 KB     | #################################

Let's vizualize the city of Toronto and neighborhoos on it. To simplify the code I use mean values of latitude and longitude of heighborhoods as the start point of the map:

In [22]:
latitude = df_neighborhood['Latitude'].mean()
longitude = df_neighborhood['Longitude'].mean()

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_neighborhood['Latitude'], df_neighborhood['Longitude'], df_neighborhood['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Next, I'm going to start utilizing the Foursquare API to explore the neighborhoods and segment them.  
  
#### Define Foursquare Credentials and Version

In [95]:
CLIENT_ID = 'WEVQCJHZEB4W2QOPCUXISUGPEB14ZDED5WSGP1TATCXL4PNR' # your Foursquare ID
CLIENT_SECRET = '2OZIDWDQUFLTEY2GUECESB11HUBKWAB1CTPVMKAHRWGCP1YK' # your Foursquare Secret
VERSION = '20191118' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

Let's create a function to generate top 100 venues within a radius of 500 meters for every neighborhoods in Toronto. I use one from lab - Segmenting and Clustering Neighborhoods in New York City:

In [96]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Using function above I create new dataframe:

In [97]:
toronto_venues = getNearbyVenues(names=df_neighborhood['Neighborhood'],
                                   latitudes=df_neighborhood['Latitude'],
                                   longitudes=df_neighborhood['Longitude'],
                                   radius=radius
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

#### Hmmm... I noticed duplicants among heighborhoods names. Let's check it:

In [98]:
df_neighborhood[df_neighborhood.duplicated(subset=['Neighborhood'],keep=False)]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
85,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
93,M9A,Queen's Park,Queen's Park,43.667856,-79.532242


We have to keep it in mind for further grouping operations! Maybe I have to use postal code column instead of neighborhood one. Check postal code column for duplicants:

In [99]:
df_neighborhood[df_neighborhood.duplicated(subset=['PostalCode'],keep=False)]

,PostalCode,Borough,Neighborhood,Latitude,Longitude


No duplicants for postal code! Let me repeat previous operation using postal code attribute and check the size of the resulting dataframe:

In [100]:
toronto_venues = getNearbyVenues(names=df_neighborhood['PostalCode'],
                                   latitudes=df_neighborhood['Latitude'],
                                   longitudes=df_neighborhood['Longitude'],
                                   radius=radius
                                  )

M1B
M1C
M1E
M1G
M1H
M1J
M1K
M1L
M1M
M1N
M1P
M1R
M1S
M1T
M1V
M1W
M1X
M2H
M2J
M2K
M2L
M2M
M2N
M2P
M2R
M3A
M3B
M3C
M3H
M3J
M3K
M3L
M3M
M3N
M4A
M4B
M4C
M4E
M4G
M4H
M4J
M4K
M4L
M4M
M4N
M4P
M4R
M4S
M4T
M4V
M4W
M4X
M4Y
M5A
M5B
M5C
M5E
M5G
M5H
M5J
M5K
M5L
M5M
M5N
M5P
M5R
M5S
M5T
M5V
M5W
M5X
M6A
M6B
M6C
M6E
M6G
M6H
M6J
M6K
M6L
M6M
M6N
M6P
M6R
M6S
M7A
M7R
M7Y
M8V
M8W
M8X
M8Y
M8Z
M9A
M9B
M9C
M9L
M9M
M9N
M9P
M9R
M9V
M9W


In [101]:
print(toronto_venues.shape)
toronto_venues.head()

(2237, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,M1C,43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
2,M1C,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,M1E,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,M1E,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


Let's check how many venues were returned for each neighborhood:

In [102]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
M1B,1,1,1,1,1,1
M1C,2,2,2,2,2,2
M1E,8,8,8,8,8,8
M1G,4,4,4,4,4,4
M1H,7,7,7,7,7,7
M1J,2,2,2,2,2,2
M1K,5,5,5,5,5,5
M1L,10,10,10,10,10,10
M1M,2,2,2,2,2,2


#### Hmm... 99 heighborhoods have venues. I recall that we have 103 neighborhoods. It means one neighborhood haven't venues.  
  
Let's find out how many unique categories can be curated from all the returned venues:

In [103]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 266 uniques categories.


Using *one-hot encoding* method I transform dataframe *toronto_venues* into dataframe of categoies of venues:

In [104]:
# one hot encoding
df_venue_categories = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_venue_categories['Neighborhood'] = df_venue_categories['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [df_venue_categories.columns[-1]] + list(df_venue_categories.columns[:-1])
df_venue_categories = df_venue_categories[fixed_columns]

print(df_venue_categories.shape)
df_venue_categories.head()

(2237, 266)


,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

#### What? Where is my *Neighborhood* column? It means we have category with name 'Neighborhood'. Let's check it:

In [105]:
toronto_venues[toronto_venues['Venue Category'] == 'Neighborhood']

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
299,M4E,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
422,M4M,43.659526,-79.340923,Leslieville,43.662070,-79.337856,Neighborhood
1076,M5H,43.650571,-79.384568,Downtown Toronto,43.653232,-79.385296,Neighborhood
1163,M5J,43.640816,-79.381752,Harbourfront,43.639526,-79.380688,Neighborhood


So, let me repeat *one-hot* transformation using another name for neighborhood column:

In [106]:
# one hot encoding
df_venue_categories = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_venue_categories['PostalCode'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [df_venue_categories.columns[-1]] + list(df_venue_categories.columns[:-1])
df_venue_categories = df_venue_categories[fixed_columns]

df_venue_categories.head()

,PostalCode,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1B,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

And let's examine the new dataframe size.

In [107]:
print(df_venue_categories.shape)

(2237, 267)


#### Next, let's group rows by postal code and by taking the mean of the frequency of occurrence of each category

In [108]:
df_neighborhood_categories = df_venue_categories.groupby('PostalCode').mean().reset_index()
df_neighborhood_categories

,PostalCode,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0

At this point we are ready to split our neigborhoods on several clusters.  

### Clustering heighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [109]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

neighborhood_clustering = df_neighborhood_categories.drop('PostalCode', axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=123).fit(neighborhood_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 2, 0, 0, 0, 0], dtype=int32)

Done! Add cluster labels to neighborhood categories dataframe:

In [110]:
df_neighborhood_categories.insert(0, 'Cluster Label', kmeans.labels_)
df_neighborhood_categories.head()

,Cluster Label,PostalCode,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,M1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

### Combine data together

Let's add clustering data to neghborhood dataframe for analysis purposes.  To do this I create new column with top 5 venues categories for each neghborhood and then add it to neghborhood dataframe together with cluster labels.
  
First, create function to sort the venues categories in descending order and combine list of them:

In [111]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[2:]
    # sort values for only present categories
    row_categories_sorted = row_categories.where(row_categories > 0).sort_values(ascending=False)
    try:
        result = row_categories_sorted.index.values[0:num_top_venues]
    except:
        result = row_categories_sorted.index.values
      
    return ','.join(result.tolist())

Let's create new column called 'Top 5' in clustering dataframe:

In [112]:
df_neighborhood_categories['Top 5'] = df_neighborhood_categories.apply(lambda x: return_most_common_venues(x,5), axis=1)

In [113]:
df_neighborhood_categories[['Cluster Label', 'PostalCode', 'Top 5']].head()

,Cluster Label,PostalCode,Top 5
0,0,M1B,"Fast Food Restaurant,Accessories Store,Afghan Restaurant,Airport,Airport Food Court"
1,0,M1C,"Construction & Landscaping,Bar,Accessories Store,Afghan Restaurant,Airport"
2,0,M1E,"Electronics Store,Rental Car Location,Pizza Place,Mexican Restaurant,Medical Center"
3,0,M1G,"Coffee Shop,Korean Restaurant,Insurance Office,Accessories Store,Afghan Restaurant"
4,0,M1H,"Thai Restaurant,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Bank"


At this point we ready to create new dataframe for all data we have - neighborhoods, geographic coordinates, venue categories and clustering labels. Let's merge it together:

In [114]:
df_toronto = df_neighborhood.merge(df_neighborhood_categories[['Cluster Label', 'PostalCode', 'Top 5']], how='left', left_on='PostalCode', right_on='PostalCode')
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Label,Top 5
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0.0,"Fast Food Restaurant,Accessories Store,Afghan Restaurant,Airport,Airport Food Court"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0.0,"Construction & Landscaping,Bar,Accessories Store,Afghan Restaurant,Airport"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,"Electronics Store,Rental Car Location,Pizza Place,Mexican Restaurant,Medical Center"
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,"Coffee Shop,Korean Restaurant,Insurance Office,Accessories Store,Afghan Restaurant"
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,"Thai Restaurant,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Bank"


#### Finally, let's visualize the resulting clusters.  
Recall we have only 99 neghborhoods with venues. So I'll draw outliers as black point.

In [115]:
import math

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add black color for outliers
rainbow.append('#000000')

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighborhood'], df_toronto['Cluster Label']):
    if math.isnan(cluster): # outlier
        label = folium.Popup(str(poi) + ' - Outlier', parse_html=True)
        cluster = kclusters
    else:
        label = folium.Popup(str(poi) + ' - Cluster ' + str(int(cluster)), parse_html=True) 
        
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)],
        fill=True,
        fill_color=rainbow[int(cluster)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item4'></a>

## 4. Examine Clusters

Now, I can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the top 5 categories, I can then assign a name to each cluster.

### Cluster 0

In [116]:
df_toronto[df_toronto['Cluster Label'] == 0][['PostalCode','Neighborhood', 'Top 5']]

,PostalCode,Neighborhood,Top 5
0,M1B,"Rouge, Malvern","Fast Food Restaurant,Accessories Store,Afghan Restaurant,Airport,Airport Food Court"
1,M1C,"Highland Creek, Rouge Hill, Port Union","Construction & Landscaping,Bar,Accessories Store,Afghan Restaurant,Airport"
2,M1E,"Guildwood, Morningside, West Hill","Electronics Store,Rental Car Location,Pizza Place,Mexican Restaurant,Medical Center"
3,M1G,Woburn,"Coffee Shop,Korean Restaurant,Insurance Office,Accessories Store,Afghan Restaurant"
4,M1H,Cedarbrae,"Thai Restaurant,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Bank"
6,M1K,"East Birchmount Park, Ionview, Kennedy Park","Discount Store,Department Store,Coffee Shop,Bus Station,Accessories Store"
7,M1L,"Clairlea, Golden Mile, Oakridge","Bus Line,Bakery,Soccer Field,Park,Metro Station"
8,M1M,"Cliffcrest, Cliffside, Scarborough Village West","Motel,American Restaurant,Accessories Store,Afghan Restaurant,Airport"
9,M1N,"Birch Cliff, Cliffside West","Skating Rink,General Entertainment,College Stadium,Café,Accessories Store"
10,M1P,"Dorset Park, Scarborough Town Centre, Wexford Heights","Indian Restaurant,Vietnamese Restaurant,Pet Store,Chinese Restaurant,Accessories Store"


### Cluster 1

In [118]:
df_toronto[df_toronto['Cluster Label'] == 1][['PostalCode','Neighborhood', 'Top 5']]

,PostalCode,Neighborhood,Top 5
20,M2L,"Silver Hills, York Mills","Cafeteria,Accessories Store,Afghan Restaurant,Airport,Airport Food Court"


### Cluster 2

In [119]:
df_toronto[df_toronto['Cluster Label'] == 2][['PostalCode','Neighborhood', 'Top 5']]

,PostalCode,Neighborhood,Top 5
5,M1J,Scarborough Village,"Spa,Playground,Accessories Store,Afghan Restaurant,Airport"
14,M1V,"Agincourt North, L'Amoreaux East, Milliken, Steeles East","Playground,Park,Accessories Store,Afghan Restaurant,Airport"
48,M4T,"Moore Park, Summerhill East","Restaurant,Playground,Park,Gym,Accessories Store"
50,M4W,Rosedale,"Park,Trail,Playground,Accessories Store,Afghan Restaurant"


### Cluster 3

In [120]:
df_toronto[df_toronto['Cluster Label'] == 3][['PostalCode','Neighborhood', 'Top 5']]

,PostalCode,Neighborhood,Top 5
23,M2P,York Mills West,"Park,Convenience Store,Bank,Accessories Store,Afghan Restaurant"
25,M3A,Parkwoods,"Park,Food & Drink Shop,Accessories Store,Afghan Restaurant,Airport"
30,M3K,"CFB Toronto, Downsview East","Park,Airport,Accessories Store,Afghan Restaurant,Airport Food Court"
40,M4J,East Toronto,"Park,Metro Station,Convenience Store,Accessories Store,Afghan Restaurant"
44,M4N,Lawrence Park,"Swim School,Park,Dim Sum Restaurant,Bus Line,Accessories Store"
64,M5P,"Forest Hill North, Forest Hill West","Trail,Sushi Restaurant,Park,Jewelry Store,Accessories Store"
74,M6E,Caledonia-Fairbanks,"Park,Women's Store,Market,Fast Food Restaurant,Accessories Store"
90,M8X,"The Kingsway, Montgomery Road, Old Mill North","River,Park,Accessories Store,Afghan Restaurant,Airport"
98,M9N,Weston,"Park,Accessories Store,Afghan Restaurant,Airport,Airport Food Court"


### Cluster 4

In [121]:
df_toronto[df_toronto['Cluster Label'] == 4][['PostalCode','Neighborhood', 'Top 5']]

,PostalCode,Neighborhood,Top 5
32,M3M,Downsview Central,"Home Service,Food Truck,Baseball Field,Accessories Store,Afghan Restaurant"
63,M5N,Roselawn,"Home Service,Garden,Accessories Store,Afghan Restaurant,Airport"
91,M8Y,"Humber Bay, King's Mill Park, Kingsway Park South East, Mimico NE, Old Mill South, The Queensway East, Royal York South East, Sunnylea","Locksmith,Home Service,Baseball Field,Accessories Store,Afghan Restaurant"
97,M9M,"Emery, Humberlea","Baseball Field,Accessories Store,Afghan Restaurant,Airport,Airport Food Court"


#### Brief analysis enables me to define cluster names. Let's look:

| Cluster label | Name  |
| ------------- | -----:|
| 0      | Coffee shop,Bakery, Pizza Place |
| 1      | Cafeteria |
| 2 | Playground  |
| 3 | Park |
| 4 | Baseball Field, Home Service |